In [11]:
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
import pandas as pd
import gspread
from sqlalchemy import create_engine
import re

from gspread_dataframe import set_with_dataframe
from pipedrive.client import Client
from main.apps.marketing.models import ZcomProspects
gc = gspread.service_account(filename="./credentials.json")

In [12]:
import environ
env = environ.Env()
database_url = env.str(
    var="DATABASE_URL", default="postgres://postgres:@postgres:5432/postgres"
)
engine = create_engine(database_url, echo=False)
client = Client(domain='https://companydomain.pipedrive.com/')
client.set_api_token('c70f38803bb436089bce792681cd178e8459c86e')

In [13]:
url_1 = 'https://docs.google.com/spreadsheets/d/1ab4eW4deYLV-3mMgAMuafoWM3e7JjYzoLDAJ83WPuaA'
workbook = gc.open_by_url(url_1)

In [14]:
sheet_1 = workbook.worksheet("Digital MKT lead")

data_1 = sheet_1.get_all_values()

df_1 = pd.DataFrame(data_1)
# df_1.columns = df_1.iloc[0]
# df_1 = df_1[1:]
df_1 = df_1.T.set_index(0).T
col_name = df_1.columns.to_list()
df_1 = df_1.drop(columns=col_name[20:])
df_1

,Date,Day,Time,Source,Ads id,Campaign name,Ads set name,First Name - Last Name,Company Name,Email,Phone Number,Contact point,URL,Duplicate,Owner,Status,Deal id,Expected \nMonthly Rev,Expected \nOnetime Rev,Client Requirements
1,21/01/21,,,,,,,,Thaniya Group,,,Call,,No,Zcom,Meeting,331,,,
2,20/01/21,,,,,,,,Sport for life,,,Form,,Yes,CDM,Lose,,,,
3,05/02/21,,,,,,,Parinya Khongpratet,"Mogen (Thailand) Co., Ltd.",parinya@mogen.co.th,06 3216 7978 / 08 6799 2055,Email,,Yes,CDM,Lose,,,,
4,05/02/21,,,,,,,,อินนิทรี,,081-655-6511,,,Yes,Zcom,Meeting,378,,,
5,09/02/21,,,,,,,คุณวราภรณ์,"Osstem (Thailand) Co.,Ltd",,089-693-6868,Call,,No,Zcom,Meeting,385,,,
6,"Feb 13, 10:53 AM",Sat,10:53 AM,,,,,พศิน ทองเกื้อ,,phasin80@gmail.com,0992214269,FB Post,,No,Zcom,Call(filter),,,,
7,"Feb 13, 11:11 AM",Sat,11:11 AM,,,,,นายชาญชัย คำพันชนะ,สำนักงานอัยการจังหวัดเดชอุดม,chanturbo@gmail.com,0804655782,FB Post,,No,Zcom,Meeting,406,,,
8,"Feb 13, 8:25 PM",Sat,8:25 PM,,,,,Cherdpong Chiwpreecha,สำนักงานตำรวจแห่งชาติ,Interpol6348@gmail.com,0869036666,FB Post,,Yes,Zcom,Meeting,402,,,
9,"Feb 15, 8:26 AM",Mon,8:26 AM,,,,,อนุรักษ์ ศรีวรรณ,Konicaminolta,Anurak.Seesuwan@konicaminolta.com,0632096685,FB Post,,No,Zcom,Call(filter),,,,
10,Feb 15,Mon,,,,,,คุณกานต์,the prodigy group,thanaphat.in@theprodigygeneration.com,062-396-8289,FB Post,,Yes,CDM,Lose,,,,


In [15]:
def marketing_zcom():
    url = 'https://docs.google.com/spreadsheets/d/1ab4eW4deYLV-3mMgAMuafoWM3e7JjYzoLDAJ83WPuaA'
    workbook = gc.open_by_url(url)
    sheet_1 = workbook.worksheet("Digital MKT lead")

    data_1 = sheet_1.get_all_values()

    df_1 = pd.DataFrame(data_1)
    df_1 = df_1.T.set_index(0).T
    col_name = df_1.columns.to_list()
    df_1 = df_1.drop(columns=col_name[20:])

    check = engine.has_table("marketing_zcom")

    if not check:
        df_1.to_sql("marketing_zcom", engine, if_exists="replace", index=False)

    data = pd.read_sql("marketing_zcom", engine)
    data = pd.DataFrame(data)
    data.index += 1

    new_data_df = df_1[len(data):]
    df_drop_blank = new_data_df.drop(
        new_data_df[new_data_df.Status != ""].index
    )
    if len(df_drop_blank) > 0:
        print('send P pat')
        
    df_drop_new_sales = new_data_df.drop(
        new_data_df[new_data_df.Status != "Call(sale)"].index
    )
    if len(df_drop_new_sales) > 0:
#         for idx, row in df_drop_new_sales.iterrows():
#             deal_id = add_deal(row)
#             cell_con = sheet_1.range(f'Q{idx + 1}:Q{idx + 1}')
#             cell_con[0].value = deal_id
#             sheet_1.update_cells(cell_con)
        print('send MKT team sale new')
#     ------------------------------------------------------------------------------------------
    old_data_df = df_1[:len(data)]
    df_drop_change = old_data_df.drop(
        old_data_df[old_data_df.Status == data.Status].index
    )
    if len(df_drop_change) > 0:
        df_drop_change_fil = df_drop_change.drop(
            df_drop_change[df_drop_change.Status != "Call(filter)"].index
        )
        df_drop_change_sale = df_drop_change.drop(
            df_drop_change[df_drop_change.Status != "Call(sale)"].index
        )
        if len(df_drop_change_fil) > 0:
            print('send P far')

        if len(df_drop_change_sale) > 0:
            for idx, row in df_drop_change_sale.iterrows():
#                 deal_id = add_deal(row)
#                 cell_con = sheet_1.range(f'Q{idx+1}:Q{idx+1}')
#                 cell_con[0].value = deal_id
#                 sheet_1.update_cells(cell_con)
                pass
            print('send MKT team')
    check_stage(df_1[['Deal id']], sheet_1)

#     df_1.to_sql("marketing_zcom", engine, if_exists="replace", index=False)
    get_notes(df_1,sheet_1)
    
#     if len(df_1) != len(data):
#         marketing_zcom()

#     return 0

In [16]:
def add_deal(data):
    org_data = {"name": data['Company Name'] or "notset"}
    response = client.organizations.create_organization(org_data)
    org_id = response["data"]["id"]

    # create person
    person_data = {
        "name": data['First Name - Last Name'] or "notset",
        "org_id": org_id,
        "phone": [data['Phone Number']],
        "email": [data['Email']],
    }
    response = client.persons.create_person(person_data)
    person_id = response["data"]["id"]

    # add a deal
    deal_data = {
        "title": data['Company Name'] or "notset",
        "person_id": person_id,
        "org_id": org_id,
        "stage_id": 12,
    }
#     response = client.deals.create_deal(deal_data)
#     deal_id = response["data"]["id"]
    context = {
        "title": data['Company Name'] or "notset",
        "name": data['First Name - Last Name'] or "notset",
        "company": data['Company Name'] or "notset",
        "phone": [data['Phone Number']],
        "email": [data['Email']],
        "stage_id": 12,
    }

    
    note_data = {
            "content": data['Client Requirements'],
            "deal_id": deal_id,
        }
    response = client.notes.create_note(note_data)

    ZcomProspects.objects.create(
        deal_id=deal_id,
        person_id=person_id,
        org_id=org_id,
        context=context,
    )
    print(deal_id)
    return deal_id

In [17]:
def get_notes(data, sheet):
    cell_col = ['T','U','V','W','X','Y','Z']
    data = data.rename(columns={'Deal id': 'deal_id'})
    for idx,row in data.iterrows():
        if row['deal_id']:
            notes = client.notes.get_all_notes(params={"deal_id": int(row['deal_id'])})['data']
            if notes:
                for i, note in enumerate(notes):
                    cell_con = sheet.range(f'{cell_col[i]}{idx+1}:{cell_col[i]}{idx+1}')
                    text = striphtml(str(note['content']))
                    cell_con[0].value = text
                    sheet.update_cells(cell_con)


In [30]:
marketing_zcom()

0
deal_id    331
Name: 1, dtype: object
13
0
deal_id    378
Name: 4, dtype: object
70
0
deal_id    385
Name: 5, dtype: object
13
0
deal_id    406
Name: 7, dtype: object
13
0
deal_id    402
Name: 8, dtype: object
70
0
deal_id    412
Name: 11, dtype: object
13
0
deal_id    420
Name: 14, dtype: object
13
0
deal_id    430
Name: 16, dtype: object
70
0
deal_id    437
Name: 18, dtype: object
70


In [29]:
def check_stage(data, sheet):
    stage_key = {12: "Call(sale)", 70 : 'Contact', 13 : "Meeting", 71 : "Meeting 2 rd", 14 : "Negotiations Started", 
                 15: "Waiting for confirm", 16: "Final Requirement", 17: "Fail"}
    data = data.rename(columns={'Deal id': 'deal_id'})
    for idx, deal in data.iterrows():
        if deal['deal_id']:
            print(deal)
            de = [client.deals.get_deal(deal_id=int(deal['deal_id']))['data']][0]['stage_id']
            cell_con = sheet.range(f'P{idx+1}:P{idx+1}')
            print(de)
            cell_con[0].value = stage_key[de]
            sheet.update_cells(cell_con)


In [28]:
import re
def striphtml(data):
    p = re.compile(r'<.*?>|&nbsp;')
    return p.sub('', data)

In [99]:
note_data = {
            "content": "test note 2",
            "deal_id": 423,
        }

# response = client.notes.create_note(note_data)
# print(response['data']['id'])

In [178]:
note = client.notes.get_all_notes(params={"deal_id":331})

In [179]:
df_note = pd.DataFrame(note['data'])
df_note

,id,user_id,deal_id,person_id,org_id,content,add_time,update_time,active_flag,pinned_to_deal_flag,pinned_to_person_flag,pinned_to_organization_flag,last_update_user_id,organization,person,deal,user
0,704,11611129,331,307,274,สอบถามในตัวระบบ เห็นจาก Ads แล้วสนใจ เพราะว่าจ...,2021-01-21 11:08:28,2021-01-21 11:08:28,True,False,False,False,None,{'name': 'Thaniya Group'},{'name': 'ภาวิทย์'},{'title': 'Thaniya Group - Toon'},"{'email': 'presale@codium.co', 'name': 'Sales ..."
1,714,11611129,331,307,274,Toon ( 27/01/21 ) : Called&nbsp; <div><ul><li>...,2021-01-22 04:03:18,2021-01-22 04:03:18,True,False,False,False,None,{'name': 'Thaniya Group'},{'name': 'ภาวิทย์'},{'title': 'Thaniya Group - Toon'},"{'email': 'presale@codium.co', 'name': 'Sales ..."
2,855,11611129,331,307,274,Toon ( 03/02/21 )&nbsp; <div><br></div><div>MO...,2021-02-03 07:47:28,2021-02-03 07:47:28,True,False,False,False,None,{'name': 'Thaniya Group'},{'name': 'ภาวิทย์'},{'title': 'Thaniya Group - Toon'},"{'email': 'presale@codium.co', 'name': 'Sales ..."
3,1012,11611129,331,307,274,Toon ( 18/02/21 ) : Call to follow up but cust...,2021-02-18 04:10:19,2021-02-18 04:10:19,True,False,False,False,None,{'name': 'Thaniya Group'},{'name': 'ภาวิทย์'},{'title': 'Thaniya Group - Toon'},"{'email': 'presale@codium.co', 'name': 'Sales ..."


In [180]:
# delete_note = client.notes.delete_note(note_id=422)

In [43]:
stage = client.stages.get_all_stages()['data']

In [44]:
df_st = pd.DataFrame(stage)
df_st

,id,order_nr,name,active_flag,deal_probability,pipeline_id,rotten_flag,rotten_days,add_time,update_time,pipeline_name,pipeline_deal_probability
0,1,0,Lead In,True,100,1,False,None,2020-07-09 03:30:36,2020-11-26 11:11:09,CLHM,False
1,62,1,Contact,True,100,1,False,None,2020-11-26 11:11:09,2020-11-26 11:11:09,CLHM,False
2,2,2,Meeting,True,100,1,False,None,2020-07-09 03:30:36,2020-11-26 11:11:09,CLHM,False
3,5,3,Negotiations Started (QT),True,100,1,False,None,2020-07-09 03:30:36,2020-11-26 11:11:09,CLHM,False
4,4,4,Waiting for confirmation,True,100,1,False,None,2020-07-09 03:30:36,2020-11-26 11:11:09,CLHM,False
5,35,5,In process,True,100,1,False,None,2020-07-13 03:22:41,2020-11-26 11:11:09,CLHM,False
6,3,6,Success,True,100,1,False,None,2020-07-09 03:30:36,2020-11-26 11:11:09,CLHM,False
7,6,7,Failed,True,100,1,False,None,2020-07-09 07:51:31,2020-11-26 11:11:09,CLHM,False
8,12,1,Lead In,True,100,3,False,None,2020-07-09 07:57:16,2020-07-09 07:57:16,Z.com,True
9,13,2,Meeting,True,100,3,False,None,2020-07-09 07:57:16,2020-07-09 07:57:16,Z.com,True


In [57]:
de = [client.deals.get_deal(deal_id=423)['data']]
# de_df = pd.DataFrame(de)
# de_df

In [58]:
de[0]['stage_id']

12

In [167]:
all_de = client.deals.get_all_deals(params={"limit": "100000"})['data']
all_de = pd.DataFrame(all_de)
new_df = all_de.drop(
        all_de[all_de.stage_id != 13].index
    )
new_df[['title', 'id']]

,title,id
290,Thaniya Group - Toon,331
341,Osstem deal,385
361,สำนักงานอัยการจังหวัดเดชอุดม deal,406
367,True,412


In [36]:
from collections import defaultdict

stage_1 = client.stages.get_all_stages()['data']
st_df1 = pd.DataFrame(stage_1)

stage_pipeline_list = []
stage_pipeline_dict = defaultdict(list)

df_pipeline = st_df1[['id', 'pipeline_id']]
for idx, row in df_pipeline.iterrows():
    stage_pipeline_dict[row['pipeline_id']] += [row['id']]

In [37]:
stage_pipeline_dict

defaultdict(list,
            {1: [1, 62, 2, 69, 5, 4, 35],
             3: [12, 70, 13, 71, 14, 15, 16],
             4: [18, 72, 19, 73, 20, 21, 22],
             2: [7, 74, 75, 76, 77, 9, 10],
             5: [24, 64, 25, 68, 26, 27, 65],
             10: [55, 61, 56, 67, 57, 58, 59],
             9: [49, 63, 50, 78, 52, 51, 53],
             11: [66]})